In [ ]:
import React from 'react';
import { motion } from 'framer-motion';
import { Card, CardContent, CardHeader, CardTitle } from "@/components/ui/card";
import { Button } from "@/components/ui/button";
import { Badge } from "@/components/ui/badge";
import { Progress } from "@/components/ui/progress";
import { 
  Flame, Scale, Apple, Droplet, Wheat, Candy, 
  ShieldAlert, Sparkles, TrendingUp, Check, AlertCircle 
} from 'lucide-react';
import { cn } from "@/lib/utils";

export default function FoodResults({ scanData, userProfile, onSave, onRescan }) {
  const getHealthColor = (score) => {
    if (score >= 8) return 'text-green-600';
    if (score >= 6) return 'text-yellow-600';
    return 'text-red-600';
  };

  const getHealthLabel = (score) => {
    if (score >= 8) return 'Excellent';
    if (score >= 6) return 'Good';
    if (score >= 4) return 'Moderate';
    return 'Poor';
  };

  const checkAllergyMatch = () => {
    if (!userProfile?.allergies?.length || !scanData.allergens?.length) return [];
    return scanData.allergens.filter(allergen => 
      userProfile.allergies.some(a => a.toLowerCase() === allergen.toLowerCase())
    );
  };

  const allergyMatches = checkAllergyMatch();

  const MacroCard = ({ icon: Icon, label, value, unit, color, target }) => (
    <Card className="border-0 shadow-sm">
      <CardContent className="p-4">
        <div className="flex items-center gap-3">
          <div className={cn("p-3 rounded-xl", color)}>
            <Icon className="w-5 h-5 text-white" />
          </div>
          <div className="flex-1">
            <p className="text-xs text-slate-500">{label}</p>
            <p className="text-xl font-bold text-slate-900">{value}{unit}</p>
            {target && (
              <Progress 
                value={(value / target) * 100} 
                className="h-1.5 mt-1"
              />
            )}
          </div>
        </div>
      </CardContent>
    </Card>
  );

  return (
    <motion.div
      initial={{ opacity: 0, y: 20 }}
      animate={{ opacity: 1, y: 0 }}
      className="space-y-4"
    >
      {/* Food Image & Name */}
      <Card className="overflow-hidden border-0 shadow-lg">
        <div className="relative h-48">
          <img 
            src={scanData.image_url} 
            alt={scanData.food_name}
            className="w-full h-full object-cover"
          />
          <div className="absolute inset-0 bg-gradient-to-t from-black/70 to-transparent" />
          <div className="absolute bottom-0 left-0 right-0 p-6">
            <h2 className="text-2xl font-bold text-white mb-2">{scanData.food_name}</h2>
            <div className="flex items-center gap-2">
              <Badge className="bg-white/20 backdrop-blur-sm text-white border-0">
                {scanData.portion_size || 'Standard portion'}
              </Badge>
              <Badge className="bg-white/20 backdrop-blur-sm text-white border-0">
                {scanData.food_category || 'Food'}
              </Badge>
            </div>
          </div>
        </div>
      </Card>

      {/* Allergy Alert */}
      {allergyMatches.length > 0 && (
        <motion.div
          initial={{ scale: 0.95 }}
          animate={{ scale: 1 }}
          className="bg-red-50 border-2 border-red-200 rounded-2xl p-4"
        >
          <div className="flex items-start gap-3">
            <div className="p-2 bg-red-100 rounded-lg">
              <AlertCircle className="w-5 h-5 text-red-600" />
            </div>
            <div>
              <p className="font-semibold text-red-900">Allergy Warning!</p>
              <p className="text-sm text-red-700 mt-1">
                Contains: {allergyMatches.join(', ')}
              </p>
            </div>
          </div>
        </motion.div>
      )}

      {/* Health Score */}
      <Card className="border-0 shadow-lg bg-gradient-to-br from-emerald-500 to-teal-600">
        <CardContent className="p-6">
          <div className="flex items-center justify-between">
            <div>
              <p className="text-emerald-100 text-sm mb-1">Health Score</p>
              <p className="text-4xl font-bold text-white">{scanData.health_score}/10</p>
              <p className="text-emerald-100 mt-1">{getHealthLabel(scanData.health_score)}</p>
            </div>
            <div className="w-24 h-24 rounded-full bg-white/20 backdrop-blur-sm flex items-center justify-center">
              <Apple className="w-12 h-12 text-white" />
            </div>
          </div>
        </CardContent>
      </Card>

      {/* Calories */}
      <Card className="border-0 shadow-lg">
        <CardContent className="p-6">
          <div className="flex items-center gap-4">
            <div className="p-4 bg-orange-100 rounded-2xl">
              <Flame className="w-8 h-8 text-orange-600" />
            </div>
            <div>
              <p className="text-sm text-slate-500">Calories</p>
              <p className="text-3xl font-bold text-slate-900">{scanData.calories}</p>
              {userProfile?.daily_calorie_target && (
                <p className="text-xs text-slate-500 mt-1">
                  {((scanData.calories / userProfile.daily_calorie_target) * 100).toFixed(1)}% of daily goal
                </p>
              )}
            </div>
          </div>
        </CardContent>
      </Card>

      {/* Macronutrients */}
      <div className="grid grid-cols-2 gap-3">
        <MacroCard 
          icon={Scale} 
          label="Protein" 
          value={scanData.protein} 
          unit="g"
          color="bg-blue-500"
          target={userProfile?.daily_protein_target}
        />
        <MacroCard 
          icon={Wheat} 
          label="Carbs" 
          value={scanData.carbs} 
          unit="g"
          color="bg-amber-500"
          target={userProfile?.daily_carbs_target}
        />
        <MacroCard 
          icon={Droplet} 
          label="Fats" 
          value={scanData.fats} 
          unit="g"
          color="bg-purple-500"
          target={userProfile?.daily_fats_target}
        />
        <MacroCard 
          icon={Candy} 
          label="Sugar" 
          value={scanData.sugar || 0} 
          unit="g"
          color="bg-pink-500"
        />
      </div>

      {/* Additional Nutrients */}
      {(scanData.fiber || scanData.sodium) && (
        <Card className="border-0 shadow-sm">
          <CardHeader>
            <CardTitle className="text-base">Additional Info</CardTitle>
          </CardHeader>
          <CardContent className="space-y-2">
            {scanData.fiber && (
              <div className="flex justify-between items-center">
                <span className="text-sm text-slate-600">Fiber</span>
                <span className="text-sm font-semibold">{scanData.fiber}g</span>
              </div>
            )}
            {scanData.sodium && (
              <div className="flex justify-between items-center">
                <span className="text-sm text-slate-600">Sodium</span>
                <span className="text-sm font-semibold">{scanData.sodium}mg</span>
              </div>
            )}
          </CardContent>
        </Card>
      )}

      {/* Allergens */}
      {scanData.allergens?.length > 0 && (
        <Card className="border-0 shadow-sm">
          <CardHeader>
            <CardTitle className="text-base flex items-center gap-2">
              <ShieldAlert className="w-4 h-4" />
              Allergens
            </CardTitle>
          </CardHeader>
          <CardContent>
            <div className="flex flex-wrap gap-2">
              {scanData.allergens.map((allergen, idx) => (
                <Badge 
                  key={idx}
                  variant="outline"
                  className={allergyMatches.includes(allergen) ? 'border-red-500 text-red-700' : ''}
                >
                  {allergen}
                </Badge>
              ))}
            </div>
          </CardContent>
        </Card>
      )}

      {/* AI Suggestions */}
      {scanData.suggestions?.length > 0 && (
        <Card className="border-0 shadow-sm">
          <CardHeader>
            <CardTitle className="text-base flex items-center gap-2">
              <Sparkles className="w-4 h-4 text-emerald-600" />
              AI Recommendations
            </CardTitle>
          </CardHeader>
          <CardContent className="space-y-3">
            {scanData.suggestions.map((suggestion, idx) => (
              <div key={idx} className="flex items-start gap-3">
                <div className="mt-0.5">
                  <TrendingUp className="w-4 h-4 text-emerald-600" />
                </div>
                <p className="text-sm text-slate-700">{suggestion}</p>
              </div>
            ))}
          </CardContent>
        </Card>
      )}

      {/* Action Buttons */}
      <div className="flex gap-3 pt-4">
        <Button 
          onClick={onSave}
          className="flex-1 h-12 bg-gradient-to-r from-emerald-600 to-teal-600 hover:from-emerald-700 hover:to-teal-700"
        >
          <Check className="w-5 h-5 mr-2" />
          Save to Diary
        </Button>
        <Button 
          onClick={onRescan}
          variant="outline"
          className="h-12 px-6"
        >
          Rescan
        </Button>
      </div>
    </motion.div>
  );
}